# Long-short-term-Memory Mixture Density Network (LSTM-MDN) for Value-at-Risk forecasting


## Dependencies

In [1]:
"""
Dependencies: Packages
"""

!pip install keras-mdn-layer # installing MDN layer for keras API in Google Colab

# import the common
import numpy as np
import pandas as pd

# reproducibilty imports
import random as rn
import os

# import TensorFlow and Keras API
import tensorflow as tf
from tensorflow_probability import distributions as tfd
from tensorflow.keras.layers import Input, InputLayer, Dense, Activation, Concatenate, LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow_probability import distributions as tfd
import keras
from keras.models import Sequential
import mdn # keras-mdn-layer

# TF callbacks
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau
tensorboard = TensorBoard(log_dir='tbdir/', histogram_freq=0, write_graph=True, write_images=False)
mon = EarlyStopping(monitor='loss', min_delta=0, patience=5, verbose=0, mode='auto')


# Data download
from google.colab import files

  Preparing metadata (setup.py) ... done
  Created wheel for keras-mdn-layer: filename=keras_mdn_layer-0.3.0-py3-none-any.whl size=7038 sha256=06274b3c87a9d4ffd1c90ef1c89dff8d4426f1ea89386484b709e1df27825644
  Stored in directory: /root/.cache/pip/wheels/c7/66/f8/9c196fe00f45d189c9e3b12f0341aa8aa33b657eb5b669465f
Successfully built keras-mdn-layer


In [2]:
"""
REPRODUCIBILITY
"""

# seed setting & reproducibility
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(6969)
rn.seed(6969)
tf.random.set_seed(6969)

In [3]:
"""
PARAMETERS & STOCKS
"""

# list of stocks
stocks = ["^GSPC", "^FTSE", "^STOXX50E"]


# Rolling window (d)
d = 10


# start date and end date of testing period
# shall be the same as in the "initialisation.R" script
test_start_date = "2021-01-01"
test_end_date = "2022-12-31"


# indicating on wheter results shall be downloaded as .csv
save_results = True
period_appendix = "_covid"

In [4]:
"""
UTILITIES: Custom functions
"""

# function to transform data frame to input data for neural network
def df_to_X_y(df, window_size = 10):
  df_as_np = df.to_numpy()
  X = []
  y = []

  for i in range(len(df_as_np) - window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i + window_size]
    y.append(label)




  return np.array(X), np.array(y)



"""
Functions complimentaring the MDN package
"""

"""
get_mixture_loss_func_REGULARIZED
- a loss function for MDN models which include a regularization term (L2-norm)
to the negative log-likelihood.
Lambda parameter is set to 20% as default.
"""
def get_mixture_loss_func_REGULARIZED(output_dim, num_mixes, lambda_reg = 0.2):
    """Construct a loss functions for the MDN layer parametrised by number of mixtures."""
    # Construct a loss function with the right number of mixtures and outputs
    def mdn_loss_func(y_true, y_pred):
        # Reshape inputs in case this is used in a TimeDistribued layer
        y_pred = tf.reshape(y_pred, [-1, (2 * num_mixes * output_dim) + num_mixes], name='reshape_ypreds')
        y_true = tf.reshape(y_true, [-1, output_dim], name='reshape_ytrue')
        # Split the inputs into paramaters
        out_mu, out_sigma, out_pi = tf.split(y_pred, num_or_size_splits=[num_mixes * output_dim,
                                                                         num_mixes * output_dim,
                                                                         num_mixes],
                                             axis=-1, name='mdn_coef_split')
        # Construct the mixture models
        cat = tfd.Categorical(logits=out_pi)
        component_splits = [output_dim] * num_mixes
        mus = tf.split(out_mu, num_or_size_splits=component_splits, axis=1)
        sigs = tf.split(out_sigma, num_or_size_splits=component_splits, axis=1)
        coll = [tfd.MultivariateNormalDiag(loc=loc, scale_diag=scale) for loc, scale
                in zip(mus, sigs)]
        mixture = tfd.Mixture(cat=cat, components=coll)
        loss = mixture.log_prob(y_true)
        loss = tf.negative(loss)
        loss = tf.reduce_mean(loss)

        # regularization
        # L2 norm
        W = tf.math.reduce_sum(tf.math.square([out_pi]))
        loss = loss + (W * lambda_reg)

        return loss

              # Actually return the loss function
    with tf.name_scope('MDN'):
        return mdn_loss_func




"""
function for decomposing results of MDN layer
OUTPUT -> mu-array, sigma-array, pi-array
Default for components of mixture distribution is 2
"""
def get_outputs(predictions,
                   N_MIXES = 2,
                   OUTPUT_DIMS = 1):
  mus = np.apply_along_axis((lambda a: a[:N_MIXES*OUTPUT_DIMS]), 1, predictions)
  sigs = np.apply_along_axis((lambda a: a[N_MIXES*OUTPUT_DIMS:2*N_MIXES*OUTPUT_DIMS]), 1, predictions)
  pis = np.apply_along_axis((lambda a: mdn.softmax(a[-N_MIXES:])), 1, predictions)

  return(mus, sigs, pis)



"""
REQUIRES: get_outputs (custom function)
custom function for generating parameter predictions
using MDN layer
"""
def MDN_predict(model, test_data, N_MIXES = 2):

  # empty dictionary for storing predictions
  out = {"mu":[],
       "sigma":[],
       "pi":[]}


  # prediction distribution parameters
  predictions = model.predict(test_data)

  # output order is mu -> sigma -> pi
  out["mu"] = get_outputs(predictions, N_MIXES = N_MIXES)[0]
  out["sigma"] = get_outputs(predictions, N_MIXES = N_MIXES)[1]
  out["pi"] = get_outputs(predictions, N_MIXES = N_MIXES)[2]


  return(out)



"""
Function for converting predictions to a pandas.DataFrame object
which can be used for extraction as file
"""
def dataframe_converter(predictions, N_MIXES = 2):
  k = 0

  # generating empty data frames
  if N_MIXES == 2:
    out = pd.DataFrame({"mu1":[],
                      "mu2":[],
                      "sigma1":[],
                      "sigma2":[],
                      "pi1":[],
                      "pi2":[]})
  elif N_MIXES == 3:
    out = pd.DataFrame({"mu1":[],
                         "mu2":[],
                         "mu3":[],
                         "sigma1":[],
                         "sigma2":[],
                         "sigma3":[],
                         "pi1":[],
                         "pi2":[],
                         "pi3":[]})


  # filling data frame with predictions
  for j in ["mu", "sigma", "pi"]:
    for i in range(0,N_MIXES):
      out.iloc[:,k] = predictions[j][:,i]
      k = k+1

  return out



## Parameters
Defining parameters and list for stock data

In [5]:
"""
Data Download
"""

# file path to the Github repository where data files are stored
file_path = 'https://raw.githubusercontent.com/NicoHerrig95/Quant_Risk_Models/main/data_files/'


# storing raw data from github repo
Data = {} # empty dictionary
for x in stocks:
    # storing each file in dictionary
    Data[x] = pd.read_csv(file_path + x + ".csv")
    # removing redundand columns
    Data[x].drop(columns = ["Unnamed: 0"], axis=1, inplace=True)
    Data[x].date = pd.to_datetime(Data[x].date)


In [6]:
"""
Data Transformation
"""


%time
# function for splitting a data frame into train set, test set and evaluation set
def data_transform_split(df,
                         test_start,
                         test_end,
                         window_size = d):



  # calculating actual start for start date
  test_start_index = df[(df['date'] > test_start_date)].index.min() # original start index
  test_start_index = test_start_index - d # adjusted start index for batch size
  test_start_date_new =  df['date'].iloc[test_start_index]

  test_data = df[(df['date'] >= test_start_date_new) & (df['date'] <= test_end_date)]
  dates_test = test_data["date"].copy()

  X_test, y_test = df_to_X_y(df = test_data["R"],
                            window_size=d)


  end_valid_index = df[(df['date'] < test_start_date_new)].index.max()
  end_train_index = int(np.floor(end_valid_index * 0.9))

  train_data = df.iloc[1:end_train_index].reset_index(drop=True)

  X_train, y_train = df_to_X_y(df = train_data["R"],
                              window_size = d)

  validation_data = df.iloc[end_train_index:end_valid_index].reset_index(drop=True)

  X_validation, y_validation = df_to_X_y(df = validation_data["R"],
                              window_size = d)


  return X_train, y_train, X_validation, y_validation, X_test, y_test







CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [7]:
# splitting thet data -> generating train/test/evaluation sets


[X_train_GSPC,
y_train_GSPC,
X_validation_GSPC,
y_validation_GSPC,
X_test_GSPC,
y_test_GSPC] = data_transform_split(df = Data["^GSPC"],
                                      test_start = test_start_date,
                                      test_end=test_end_date)

[X_train_FTSE,
y_train_FTSE,
X_validation_FTSE,
y_validation_FTSE,
X_test_FTSE,
y_test_FTSE] = data_transform_split(df = Data["^FTSE"],
                                      test_start = test_start_date,
                                      test_end=test_end_date)


[X_train_EUSTOXX,
y_train_EUSTOXX,
X_validation_EUSTOXX,
y_validation_EUSTOXX,
X_test_EUSTOXX,
y_test_EUSTOXX] = data_transform_split(df = Data["^STOXX50E"],
                                      test_start = test_start_date,
                                      test_end=test_end_date)




## LSTM-Mixed Density Network

### Model Building
1. LSTM-MDN with 2-component MDN layer
2. LSTM-MDN with 2-component MDN layer & regularized loss function
3. LSTM-MDN with 3-component MDN layer


In [8]:
"""
Model 1: 2-component MDN (vanilla model)
Architecture
6 Neurons in LSTM Layer
10 Neurons in Hidden Layer
2 components for Gaussian Mixture

Loss function: mean negative log-likelihood loss of y given the mixture parameters
Optimizer: Adam with learning rate of 0.001
"""


"S&P500"
model_plain_GSPC = Sequential()
model_plain_GSPC.add(InputLayer(input_shape = (d,1)))
model_plain_GSPC.add(LSTM(6, activation = "relu"))
model_plain_GSPC.add(Dense(10, activation = "relu"))
model_plain_GSPC.add(mdn.MDN(output_dimension = 1,
                  num_mixtures = 2))

# compiling model
# using neg. log lik as loss
model_plain_GSPC.compile(loss=mdn.get_mixture_loss_func(1,2),
                         optimizer=keras.optimizers.Adam())


"FTSE"
model_plain_FTSE = Sequential()
model_plain_FTSE.add(InputLayer(input_shape = (d,1)))
model_plain_FTSE.add(LSTM(6, activation = "relu"))
model_plain_FTSE.add(Dense(10, activation = "relu"))
model_plain_FTSE.add(mdn.MDN(output_dimension = 1,
                  num_mixtures = 2))

# compiling model
# using neg. log lik as loss
model_plain_FTSE.compile(loss=mdn.get_mixture_loss_func(1,2),
                         optimizer=keras.optimizers.Adam())

"EUROSTOXX50"
model_plain_EUSTOXX = Sequential()
model_plain_EUSTOXX.add(InputLayer(input_shape = (d,1)))
model_plain_EUSTOXX.add(LSTM(6, activation = "relu"))
model_plain_EUSTOXX.add(Dense(10, activation = "relu"))
model_plain_EUSTOXX.add(mdn.MDN(output_dimension = 1,
                  num_mixtures = 2))

# compiling model
# using neg. log lik as loss
model_plain_EUSTOXX.compile(loss=mdn.get_mixture_loss_func(1,2),
                         optimizer=keras.optimizers.Adam())


In [9]:
"""
Model 2: 2-component MDN w/ regularized loss function
Architecture
6 Neurons in LSTM Layer
10 Neurons in Hidden Layer
2 components for Gaussian Mixture

Loss function: REGULARIZED (L2-norm) mean negative log-likelihood loss of y given the mixture parameters
Optimizer: Adam with learning rate of 0.001
"""

"S&P500"
model_regularized_GSPC = Sequential()
model_regularized_GSPC.add(InputLayer(input_shape = (d,1)))
model_regularized_GSPC.add(LSTM(6, activation = "relu"))
model_regularized_GSPC.add(Dense(10, activation = "relu"))
model_regularized_GSPC.add(mdn.MDN(output_dimension = 1,
                  num_mixtures = 2))


# compiling model
# using REGULARIZED neg. log lik as loss
model_regularized_GSPC.compile(loss=get_mixture_loss_func_REGULARIZED(1,2, lambda_reg=0.1),
                         optimizer=keras.optimizers.Adam())

"FTSE"
model_regularized_FTSE = Sequential()
model_regularized_FTSE.add(InputLayer(input_shape = (d,1)))
model_regularized_FTSE.add(LSTM(6, activation = "relu"))
model_regularized_FTSE.add(Dense(10, activation = "relu"))
model_regularized_FTSE.add(mdn.MDN(output_dimension = 1,
                  num_mixtures = 2))

# compiling model
# using REGULARIZED neg. log lik as loss
model_regularized_FTSE.compile(loss=get_mixture_loss_func_REGULARIZED(1,2, lambda_reg=0.1),
                         optimizer=keras.optimizers.Adam())


"EUROSTOXX50"
model_regularized_EUSTOXX = Sequential()
model_regularized_EUSTOXX.add(InputLayer(input_shape = (d,1)))
model_regularized_EUSTOXX.add(LSTM(6, activation = "relu"))
model_regularized_EUSTOXX.add(Dense(10, activation = "relu"))
model_regularized_EUSTOXX.add(mdn.MDN(output_dimension = 1,
                  num_mixtures = 2))


# compiling model
# using REGULARIZED neg. log lik as loss
model_regularized_EUSTOXX.compile(loss=get_mixture_loss_func_REGULARIZED(1,2, lambda_reg=0.1),
                         optimizer=keras.optimizers.Adam())


In [10]:
"""
Model 3: 3-component MDN
Architecture
6 Neurons in LSTM Layer
10 Neurons in Hidden Layer
3 components for Gaussian Mixture

Loss function: mean negative log-likelihood loss of y given the mixture parameters
Optimizer: Adam with learning rate of 0.001
"""



"GSPC"
model_C3_GSPC = Sequential()
model_C3_GSPC.add(InputLayer(input_shape = (d,1)))
model_C3_GSPC.add(LSTM(6, activation = "relu"))
model_C3_GSPC.add(Dense(10, activation = "relu"))
model_C3_GSPC.add(mdn.MDN(output_dimension = 1,
                  num_mixtures = 3))


# compiling model
model_C3_GSPC.compile(loss=mdn.get_mixture_loss_func(1, 3),
                         optimizer=keras.optimizers.Adam())

"FTSE"
model_C3_FTSE = Sequential()
model_C3_FTSE.add(InputLayer(input_shape = (d,1)))
model_C3_FTSE.add(LSTM(6, activation = "relu"))
model_C3_FTSE.add(Dense(10, activation = "relu"))
model_C3_FTSE.add(mdn.MDN(output_dimension = 1,
                  num_mixtures = 3))

# compiling model
model_C3_FTSE.compile(loss=mdn.get_mixture_loss_func(1, 3),
                         optimizer=keras.optimizers.Adam())





"EUROSTOXX 50"
model_C3_EUSTOXX = Sequential()
model_C3_EUSTOXX.add(InputLayer(input_shape = (d,1)))
model_C3_EUSTOXX.add(LSTM(6, activation = "relu"))
model_C3_EUSTOXX.add(Dense(10, activation = "relu"))
model_C3_EUSTOXX.add(mdn.MDN(output_dimension = 1,
                  num_mixtures = 3))

# compiling model
model_C3_EUSTOXX.compile(loss=mdn.get_mixture_loss_func(1, 3),
                         optimizer=keras.optimizers.Adam())




Model Training



In [11]:
"fitting models for S&P500"

"vanilla model"
model_plain_GSPC.fit(X_train_GSPC, y_train_GSPC,
                     validation_data=(X_validation_GSPC,y_validation_GSPC),
                     epochs = 100,
                     callbacks = [mon, tensorboard],
                     verbose = 0)

"regularized model"
model_regularized_GSPC.fit(X_train_GSPC, y_train_GSPC,
                           validation_data=(X_validation_GSPC,y_validation_GSPC),
                           epochs = 100,
                           callbacks = [mon, tensorboard],
                            verbose = 0)

"3-component model"
model_C3_GSPC.fit(X_train_GSPC, y_train_GSPC,
                  validation_data=(X_validation_GSPC,y_validation_GSPC),
                  epochs = 100,
                  callbacks = [mon, tensorboard],
                  verbose = 0)

In [12]:
"fitting models for FTSE"

"vanilla model"
model_plain_FTSE.fit(X_train_FTSE, y_train_FTSE,
                     validation_data=(X_validation_FTSE,y_validation_FTSE),
                     epochs = 100,
                     callbacks = [mon, tensorboard],
                     verbose = 0)

"regularized model"
model_regularized_FTSE.fit(X_train_FTSE, y_train_FTSE,
                           validation_data=(X_validation_FTSE,y_validation_FTSE),
                           epochs = 100,
                           callbacks = [mon, tensorboard],
                            verbose = 0)

"3-component model"
model_C3_FTSE.fit(X_train_FTSE, y_train_FTSE,
                  validation_data=(X_validation_FTSE,y_validation_FTSE),
                  epochs = 100,
                  callbacks = [mon, tensorboard],
                  verbose = 0)

In [13]:
"fitting models for EUROSTOXX50"

"vanilla model"
model_plain_EUSTOXX.fit(X_train_EUSTOXX, y_train_EUSTOXX,
                     validation_data=(X_validation_EUSTOXX,y_validation_EUSTOXX),
                     epochs = 100,
                     callbacks = [mon, tensorboard],
                     verbose = 0)

"regularized model"
model_regularized_EUSTOXX.fit(X_train_EUSTOXX, y_train_EUSTOXX,
                           validation_data=(X_validation_EUSTOXX,y_validation_EUSTOXX),
                           epochs = 100,
                           callbacks = [mon, tensorboard],
                            verbose = 0)

"3-component model"
model_C3_EUSTOXX.fit(X_train_EUSTOXX, y_train_EUSTOXX,
                  validation_data=(X_validation_EUSTOXX,y_validation_EUSTOXX),
                  epochs = 100,
                  callbacks = [mon, tensorboard],
                  verbose = 0)

Model Predictions

In [14]:
# running predictions and storing results in a data frame

import warnings
warnings.filterwarnings("ignore")

"S&P 500"
GSPC_vanilla = dataframe_converter(predictions = MDN_predict(model = model_plain_GSPC,
                                       test_data = X_test_GSPC,
                                       N_MIXES=2),
                                   N_MIXES = 2)


GSPC_regularized = dataframe_converter(predictions = MDN_predict(model = model_regularized_GSPC,
                                       test_data = X_test_GSPC,
                                       N_MIXES=2),
                                   N_MIXES = 2)


GSPC_C3 = dataframe_converter(predictions = MDN_predict(model = model_C3_GSPC,
                                       test_data = X_test_GSPC,
                                       N_MIXES=3),
                                   N_MIXES = 3)





"FTSE"
FTSE_vanilla = dataframe_converter(predictions = MDN_predict(model = model_plain_FTSE,
                                       test_data = X_test_FTSE,
                                       N_MIXES=2),
                                   N_MIXES = 2)


FTSE_regularized = dataframe_converter(predictions = MDN_predict(model = model_regularized_FTSE,
                                       test_data = X_test_FTSE,
                                       N_MIXES=2),
                                   N_MIXES = 2)


FTSE_C3 = dataframe_converter(predictions = MDN_predict(model = model_C3_FTSE,
                                       test_data = X_test_FTSE,
                                       N_MIXES=3),
                                   N_MIXES = 3)




"EUROSTOXX50"
EUSTOXX_vanilla = dataframe_converter(predictions = MDN_predict(model = model_plain_EUSTOXX,
                                       test_data = X_test_EUSTOXX,
                                       N_MIXES=2),
                                   N_MIXES = 2)


EUSTOXX_regularized = dataframe_converter(predictions = MDN_predict(model = model_regularized_EUSTOXX,
                                       test_data = X_test_EUSTOXX,
                                       N_MIXES=2),
                                   N_MIXES = 2)


EUSTOXX_C3 = dataframe_converter(predictions = MDN_predict(model = model_C3_EUSTOXX,
                                       test_data = X_test_EUSTOXX,
                                       N_MIXES=3),
                                   N_MIXES = 3)

16/16 [==============================] - 0s 2ms/step


In [15]:
"""
DATA DOWNLOAD
"""

# if save_results == True, results data will be downloaded to local drive
if save_results == True:

  # results S&P
  GSPC_vanilla.to_csv('GSPC_vanilla'+ period_appendix +'.csv', encoding = 'utf-8-sig')
  files.download('GSPC_vanilla'+ period_appendix +'.csv')

  GSPC_regularized.to_csv('GSPC_regularized'+ period_appendix +'.csv', encoding = 'utf-8-sig')
  files.download('GSPC_regularized'+ period_appendix +'.csv')

  GSPC_C3.to_csv('GSPC_C3'+ period_appendix +'.csv', encoding = 'utf-8-sig')
  files.download('GSPC_C3'+ period_appendix +'.csv')

  # results FTSE
  FTSE_vanilla.to_csv('FTSE_vanilla'+ period_appendix +'.csv', encoding = 'utf-8-sig')
  files.download('FTSE_vanilla'+ period_appendix +'.csv')

  FTSE_regularized.to_csv('FTSE_regularized'+ period_appendix +'.csv', encoding = 'utf-8-sig')
  files.download('FTSE_regularized'+ period_appendix +'.csv')

  FTSE_C3.to_csv('FTSE_C3'+ period_appendix +'.csv', encoding = 'utf-8-sig')
  files.download('FTSE_C3'+ period_appendix +'.csv')

  # results EUROSTOXX50
  EUSTOXX_vanilla.to_csv('EUSTOXX_vanilla'+ period_appendix +'.csv', encoding = 'utf-8-sig')
  files.download('EUSTOXX_vanilla'+ period_appendix +'.csv')

  EUSTOXX_regularized.to_csv('EUSTOXX_regularized'+ period_appendix +'.csv', encoding = 'utf-8-sig')
  files.download('EUSTOXX_regularized'+ period_appendix +'.csv')

  EUSTOXX_C3.to_csv('EUSTOXX_C3'+ period_appendix +'.csv', encoding = 'utf-8-sig')
  files.download('EUSTOXX_C3'+ period_appendix +'.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# clearing & resetting tensorflow backend
tf.keras.backend.clear_session()